In [1]:
# Importar Bibliotecas
import numpy as np
import matplotlib.pyplot as plt
from keras.datasets import mnist
from keras.models import Sequential
from keras.layers import Flatten, Dense
from keras.callbacks import EarlyStopping
from keras.utils import to_categorical
from keras.initializers import Zeros
from sklearn.model_selection import train_test_split
from sklearn.utils import resample
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_curve, auc


ModuleNotFoundError: No module named 'tensorflow'

In [ ]:

# Preparación de datos

# Cargar el conjunto de datos MNIST
(x_train, y_train), (x_test, y_test) = mnist.load_data()


# Imprimir las dimensiones de los conjuntos de entrenamiento y prueba
print("Datos de entrenamiento (x_train): ", x_train.shape)
print("Etiquetas de entrenamiento (y_train): ", y_train.shape)
print("Datos de prueba (x_test): ", x_test.shape)
print("Etiquetas de prueba (y_test): ", y_test.shape)

# Mostrar las primeras 5 imágenes del conjunto de entrenamiento con sus etiquetas
for i in range(5):
    plt.subplot(1, 5, i+1)
    plt.imshow(x_train[i], cmap='gray')
    plt.title("Etiqueta: " + str(y_train[i]))
    plt.axis('off')
plt.show()


# Mostrar el número original de imágenes por clase
print("Número original de imágenes por clase en el conjunto de entrenamiento:")
unique, counts = np.unique(y_train, return_counts=True)
print(dict(zip(unique, counts)))

# Normalizar los datos: Los valores de píxeles en imágenes en escala de grises están en el rango 0-255 (8 bits por píxel).
# Al dividir por 255.0, transformamos estos valores a un rango de 0-1 para facilitar el entrenamiento de la red neuronal.
x_train = x_train / 255.0
x_test = x_test / 255.0

# Convertir las etiquetas a binario: 1 para 'cinco' y 0 para 'no cinco'
y_train_bin = (y_train == 5).astype(np.float32)
y_test_bin = (y_test == 5).astype(np.float32)

# Balancear el conjunto de entrenamiento para tener aproximadamente el mismo número de cincos y no cincos
x_cincos = x_train[y_train_bin == 1]
y_cincos = y_train_bin[y_train_bin == 1]
x_no_cincos = x_train[y_train_bin == 0][:len(x_cincos)]
y_no_cincos = y_train_bin[y_train_bin == 0][:len(y_cincos)]

# * Concatenación de arrays con cincos y otros números
x_train_balanced = np.concatenate((x_cincos, x_no_cincos), axis=0)
y_train_balanced = np.concatenate((y_cincos, y_no_cincos), axis=0)

# Verificación del balanceo
print("\nDespués del balanceo:")
print("Número de 'cincos':", np.sum(y_train_balanced == 1))
print("Número de 'no cincos':", np.sum(y_train_balanced == 0))

# Mezclar los datos balanceados
x_train_balanced, y_train_balanced = resample(x_train_balanced, y_train_balanced)

# Mostrar el número despues del balanceo de imágenes por clase
print("Número original de imágenes por clase en el conjunto de entrenamiento:")
unique, counts = np.unique(y_train_balanced, return_counts=True)
print("Recuerda que 1.0 es un 5 y 0.0 es cualquier otro número")
print(dict(zip(unique, counts)))

# Aplanar las imágenes para la entrada a la red neuronal
x_train_balanced = x_train_balanced.reshape((x_train_balanced.shape[0], -1))
x_test = x_test.reshape((x_test.shape[0], -1))

# Mostrar dimensiones finales de los conjuntos de datos
print("\nDimensiones finales:")
print("x_train_balanced:", x_train_balanced.shape)
print("y_train_balanced:", y_train_balanced.shape)
print("x_test:", x_test.shape)
print("y_test_bin:", y_test_bin.shape)

# Dividir los datos de entrenamiento balanceados en conjuntos de entrenamiento y validación
x_train_balanced, x_val_balanced, y_train_balanced, y_val_balanced = train_test_split(
    x_train_balanced, y_train_balanced, test_size=0.2, random_state=42)

# Mostrar las dimensiones de los nuevos conjuntos de entrenamiento y validación
print("Dimensiones de entrenamiento balanceado: ", x_train_balanced.shape, y_train_balanced.shape)
print("Dimensiones de validación balanceado: ", x_val_balanced.shape, y_val_balanced.shape)

In [ ]:
# Construcción de la Red Neuronal
model = Sequential([
    Dense(2, input_shape=(784,), activation='relu'),
    Dense(1, activation='sigmoid')
])

# Compilación del modelo
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
history = model.fit(x_train_bal, y_train_bal, epochs=50, batch_size=32,
                    validation_data=(x_val_bal, y_val_bal), callbacks=[early_stopping])

In [ ]:
# Evaluación del Modelo


# Evaluar el modelo en el conjunto de prueba
test_loss, test_acc = model.evaluate(x_test, y_test_bin, verbose=2)
predictions = model.predict(x_test).flatten()
predictions_classes = (predictions > 0.5).astype(int)

# Graficar el historial de entrenamiento
plt.figure(figsize=(14, 7))
plt.subplot(1, 2, 1)
plt.plot(history.history['accuracy'], label='Accuracy de Entrenamiento')
plt.plot(history.history['val_accuracy'], label='Accuracy de Validación')
plt.title('Accuracy durante el Entrenamiento')
plt.legend()

plt.subplot(1, 2, 2)
plt.plot(history.history['loss'], label='Loss de Entrenamiento')
plt.plot(history.history['val_loss'], label='Loss de Validación')
plt.title('Loss durante el Entrenamiento')
plt.legend()
plt.show()

# Mostrar imágenes de errores de clasificación (Omitido por brevedad, implementa un loop para mostrar imágenes)

# Reportar métricas de clasificación
print(classification_report(y_test_bin, predictions_classes))

# Curva ROC y AUC
fpr, tpr, thresholds = roc_curve(y_test_bin, predictions)
roc_auc = auc(fpr, tpr)

plt.figure()
plt.plot(fpr, tpr, color='darkorange', lw=2, label='ROC curve (area = %0.2f)' % roc_auc)
plt.plot([0, 1], [0, 1], color='navy', lw=2, linestyle='--')
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.05])
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('Curva ROC')
plt.legend(loc="lower right")
plt.show()

# Determinar el mejor umbral basado en algún criterio, por ejemplo, el punto más cercano a (0,1) en la curva ROC
